In [3]:
pip install requests beautifulsoup4 pandas

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 5.1 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.5 MB 3.7 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.5 MB 4.4 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/11.5 MB 4.6 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.5 MB 4.8 MB/s eta 0:00:02
   --------------------- -----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:


import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://www.procyclingstats.com"
#RACE_BASE = "/race/vuelta-a-espana/2024/"
RACE_BASE = "/race/vuelta-a-espana/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Script para extraer resultados de la Vuelta a España 2015-2024
anios = [f"{i}/" for i in range(2015, 2025)]  # Años de 2015 a 2024

# Asumimos 21 etapas
etapas = [f"stage-{i}" for i in range(1, 22)]


all_etapas = []

#recorrer cada año y cada etapa
for anio in anios:
    print(f"Procesando año: {anio.strip('/')}")

    for etapa in etapas:
        url = BASE_URL + RACE_BASE + anio + etapa
        print(f"Procesando: {url}")
        
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Error al acceder a la etapa {etapa}: {response.status_code}")
            continue
        
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "results"})

        if not table:
            print(f"No se encontró tabla en {etapa}")
            continue

        headers = [th.text.strip() for th in table.find_all("th")]
        rows = []
        for tr in table.find_all("tr")[1:]:
            cols = [td.text.strip() for td in tr.find_all("td")]
            if cols:
                rows.append(cols)

        df_etapa = pd.DataFrame(rows, columns=headers)
        df_etapa["Etapa"] = etapa
        df_etapa["Año"] = anio.strip("/")
        all_etapas.append(df_etapa)

        # Esperar para evitar bloqueo
        time.sleep(3)

# Unir todas las etapas
df_total = pd.concat(all_etapas, ignore_index=True)
print(df_total)
# Guardar como CSV
df_total.to_csv("vuelta_espana_2015_2024_etapas.csv", index=False)
print("✅ Archivo CSV generado: vuelta_espana_2015_2024_etapas.csv")

Procesando año: 2015
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-1
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-2
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-3
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-4
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-5
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-6
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-7
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-8
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-9
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-10
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-11
Procesando: https://www.procyclingstats.com/race/vuelta-a-espana/2015/stage-12
Procesando: https://www.procyclingstats.

In [4]:
anios = [f"{i}/" for i in range(2015, 2025)]
print("Años procesados:", anios)

Años procesados: ['2015/', '2016/', '2017/', '2018/', '2019/', '2020/', '2021/', '2022/', '2023/', '2024/']
